# Previsão de Topos e Fundos no Bitcoin
Este notebook apresenta o fluxo completo do projeto, desde a coleta de dados até a avaliação de modelos de Machine Learning.

## Objetivo
Identificar pontos de reversão (Topos e Fundos) no preço do Bitcoin utilizando indicadores técnicos e modelos supervisionados. Focamos na comparação entre Regressão Logística, Random Forest e Redes Neurais.

> **Dica**: Se você receber erro de `ModuleNotFoundError`, verifique se o Kernel do Python selecionado no VS Code (canto superior direito) é o mesmo onde as bibliotecas foram instaladas.

## 1. Coleta de Dados
Utilizamos a API pública da Binance para obter dados históricos.

In [ ]:
import pandas as pd
import os
from data_handler import get_historical_data

data_file = 'data/btc_historical_1h.parquet'
if not os.path.exists(data_file):
    df = get_historical_data(symbol='BTCUSDT', interval='1h', days=365)
    df.to_parquet(data_file, index=False)
else:
    df = pd.read_parquet(data_file)

print(f"Dados carregados: {len(df)} registros.")
df.head()

## 2. Engenharia de Atributos e Rotulagem
Calculamos indicadores (RSI, MACD, etc) e definimos Topos e Fundos como máximos/mínimos em uma janela de 24h.

In [ ]:
from feature_engineering import apply_feature_engineering, label_tops_bottoms

df_features = apply_feature_engineering(df.copy())
df_labeled = label_tops_bottoms(df_features, window=24)
df_labeled = df_labeled.dropna()

print("Distribuição de Rótulos (0=Nada, 1=Topo, 2=Fundo):")
print(df_labeled['label'].value_counts())
df_labeled.head()

## 3. Treinamento e Avaliação Completa
Comparamos os três modelos e geramos as matrizes de confusão.

In [ ]:
from model_trainer import train_eval_models, plot_confusion_matrix, plot_predictions_on_price
import matplotlib.pyplot as plt

results, y_test, test_dates, test_close = train_eval_models(df_labeled)

for model_name, metrics in results.items():
    print(f"\n--- {model_name} ---")
    print(f"Acurácia: {metrics['accuracy']:.4f}")
    print(f"F1-Score: {metrics['f1_macro']:.4f}")
    
    # Mostrar Matriz de Confusão
    plot_confusion_matrix(y_test, metrics['y_pred'], model_name)
    plt.show()

## 4. Visualização de Previsões no Gráfico de Preço
Abaixo visualizamos onde o modelo (ex: Neural Network) previu os Topos e Fundos.

In [ ]:
# Visualizando para a Rede Neural especificamente
nn_preds = results['Neural Network']['y_pred']
plot_predictions_on_price(test_dates, test_close, y_test, nn_preds, 'Neural Network')
plt.show()